In [2]:
import fiona
import geopandas as gpd
import pandas as pd

In [3]:
ptv_layers = fiona.listlayers('../data/raw/PTV DATA/ll_gda94/filegdb/whole_of_dataset/victoria/PUBLIC_TRANSPORT.gdb')
vmadmin_layers = fiona.listlayers('../data/raw/VIC admin/ll_gda2020/filegdb/whole_of_dataset/victoria/VMADMIN.gdb')

gdf_AD_LOCALITY_AREA = gpd.read_file('../data/raw/VIC admin/ll_gda2020/filegdb/whole_of_dataset/victoria/VMADMIN.gdb', layer='AD_LOCALITY_AREA_POLYGON')
gdf_vic = gdf_AD_LOCALITY_AREA[gdf_AD_LOCALITY_AREA['STATE'] == 'VIC']

### tram

In [4]:
gdf_METRO_TRAM = gpd.read_file('../data/raw/PTV DATA/ll_gda94/filegdb/whole_of_dataset/victoria/PUBLIC_TRANSPORT.gdb', 
layer='PTV_METRO_TRAM_STOP')

df_vic_TRAM = pd.DataFrame(gdf_METRO_TRAM)

df_vic_TRAM['suburb'] = df_vic_TRAM['STOP_NAME'].str.extract(r'(\(.*\))')
df_vic_TRAM['suburb'] = df_vic_TRAM['suburb'].str.extract(r'([^\(\)]+)')

# df_vic_TRAM.to_csv('processed_csv/tram.csv')

### train

In [5]:
gdf_REGIONAL_TRAIN= gpd.read_file('../data/raw/PTV DATA/ll_gda94/filegdb/whole_of_dataset/victoria/PUBLIC_TRANSPORT.gdb', 
layer='PTV_REGIONAL_TRAIN_STATION')

gdf_METRO_TRAIN = gpd.read_file('../data/raw/PTV DATA/ll_gda94/filegdb/whole_of_dataset/victoria/PUBLIC_TRANSPORT.gdb', 
layer='PTV_METRO_TRAIN_STATION')

df_mtrain_stops = pd.DataFrame({'STOP_ID': gdf_METRO_TRAIN['STOP_ID'], 'STOP_NAME': gdf_METRO_TRAIN['STOP_NAME'], 
'geometry': gdf_METRO_TRAIN['geometry'], 'type': pd.Series(["METRO_TRAIN" for x in range(len(gdf_METRO_TRAIN.index))])})

df_rtrain_stops = pd.DataFrame({'STOP_ID': gdf_REGIONAL_TRAIN['STOP_ID'], 'STOP_NAME': gdf_REGIONAL_TRAIN['STOP_NAME'],
'geometry': gdf_REGIONAL_TRAIN['geometry'], 'type': pd.Series(["REGIONAL_TRAIN" for x in range(len(gdf_REGIONAL_TRAIN.index))])})

df_train_stops = pd.concat([df_mtrain_stops, df_rtrain_stops], axis=0)

df_train_stops['suburb'] = df_train_stops['STOP_NAME'].str.extract(r'(\(.*\))')
df_train_stops['suburb'] = df_train_stops['suburb'].str.extract(r'([^\(\)]+)')

# df_train_stops.to_csv('processed_csv/train.csv')

### bus

In [6]:
gdf_REGIONAL_BUS= gpd.read_file('../data/raw/PTV DATA/ll_gda94/filegdb/whole_of_dataset/victoria/PUBLIC_TRANSPORT.gdb', 
layer='PTV_REGIONAL_BUS_STOP')

gdf_METRO_BUS= gpd.read_file('../data/raw/PTV DATA/ll_gda94/filegdb/whole_of_dataset/victoria/PUBLIC_TRANSPORT.gdb', 
layer='PTV_METRO_BUS_STOP')

gdf_SKYBUS= gpd.read_file('../data/raw/PTV DATA/ll_gda94/filegdb/whole_of_dataset/victoria/PUBLIC_TRANSPORT.gdb', 
layer='PTV_SKYBUS_STOP')

gdf_REGIONAL_COACH= gpd.read_file('../data/raw/PTV DATA/ll_gda94/filegdb/whole_of_dataset/victoria/PUBLIC_TRANSPORT.gdb', 
layer='PTV_REGIONAL_COACH_STOP')

df_REGIONAL_BUS_stops = pd.DataFrame({'STOP_ID': gdf_REGIONAL_BUS['STOP_ID'], 'STOP_NAME': gdf_REGIONAL_BUS['STOP_NAME'], 
'geometry': gdf_REGIONAL_BUS['geometry'], 'type': pd.Series(["REGIONAL_BUS" for x in range(len(gdf_REGIONAL_BUS.index))])})

df_METRO_BUS = pd.DataFrame({'STOP_ID': gdf_METRO_BUS['STOP_ID'], 'STOP_NAME': gdf_METRO_BUS['STOP_NAME'], 
'geometry': gdf_METRO_BUS['geometry'], 'type': pd.Series(["METRO_BUS" for x in range(len(gdf_METRO_BUS.index))])})

df_SKYBUS = pd.DataFrame({'STOP_ID': gdf_SKYBUS['STOP_ID'], 'STOP_NAME': gdf_SKYBUS['STOP_NAME'], 
'geometry': gdf_SKYBUS['geometry'], 'type': pd.Series(["SKYBUS" for x in range(len(gdf_SKYBUS.index))])})

df_REGIONAL_COACH = pd.DataFrame({'STOP_ID': gdf_REGIONAL_COACH['STOP_ID'], 'STOP_NAME': gdf_REGIONAL_COACH['STOP_NAME'],
'geometry': gdf_REGIONAL_COACH['geometry'], 'type': pd.Series(["REGIONAL_COACH" for x in range(len(gdf_REGIONAL_COACH.index))])})

df_bus_stops = pd.concat([df_REGIONAL_BUS_stops, df_METRO_BUS, df_SKYBUS, df_REGIONAL_COACH], axis=0)

df_bus_stops['suburb'] = df_bus_stops['STOP_NAME'].str.extract(r'(\(.*\))')
df_bus_stops['suburb'] = df_bus_stops['suburb'].str.extract(r'([^\(\)]+)')

### stations counts

In [7]:
# convert suburb name to uppercase

df_bus_stops['suburb'] = df_bus_stops['suburb'].astype('string')
df_bus_stops['suburb'] = df_bus_stops['suburb'].str.upper()

df_train_stops['suburb'] = df_train_stops['suburb'].astype('string')
df_train_stops['suburb'] = df_train_stops['suburb'].str.upper()

df_vic_TRAM['suburb'] = df_vic_TRAM['suburb'].astype('string')
df_vic_TRAM['suburb'] = df_vic_TRAM['suburb'].str.upper()

In [8]:
# df_station_count = pd.DataFrame(gdf_vic['NAME'])
# df_station_count['NAME'] = df_station_count['NAME'].astype("string")

In [9]:
train_count = pd.DataFrame(
    {
        'NAME': df_train_stops['suburb'].value_counts().index,
        'train_counts': df_train_stops['suburb'].value_counts().values
        }
    )

tram_count = pd.DataFrame(
    {
        'NAME': df_vic_TRAM['suburb'].value_counts().index,
        'tram_counts': df_vic_TRAM['suburb'].value_counts().values
        }
    )

bus_count = pd.DataFrame(
    {
        'NAME': df_bus_stops['suburb'].value_counts().index,
        'bus_counts': df_bus_stops['suburb'].value_counts().values
        }
    )

In [35]:
bus_count[bus_count['NAME'] == 'BRIGHTON']

train_count[train_count['NAME'] == 'BRIGHTON']

,NAME,train_counts
4,BRIGHTON,4


In [36]:
from functools import reduce

df_station_count = reduce(lambda  left,right: pd.merge(left,right,on=['NAME'],
how='outer'), [train_count, tram_count, bus_count])

In [37]:
df_station_count

,NAME,train_counts,tram_counts,bus_counts
0,MELBOURNE CITY,7,129,55
1,NORTHCOTE,5,28,73
2,CAMBERWELL,5,66,51
3,RICHMOND,5,59,8
4,BRIGHTON,4,<NA>,107
...,...,...,...,...
1165,GLENROWAN,<NA>,<NA>,1
1166,TINAMBA,<NA>,<NA>,1
1167,WOOLSTHORPE,<NA>,<NA>,1
1168,COWWARR,<NA>,<NA>,1
